In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,AutoModelForSeq2SeqLM

import torch
print(torch.__version__)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())



c:\Users\Jaime Robledo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.6.0.dev20241112+cu121
True
Torch disponible: True
TensorFlow disponible: False


In [2]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [ ]:

generator = pipeline('text-generation', model="gpt2", device=0)

# Lista para almacenar las predicciones
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Clasificar cada opinión y mostrar resultado
for opinion in df['opinion']:

    prompt_base = f"Clasifica la siguiente opinión en base a como interpretes la actitud del mensaje, dicha actitud puede ser vista como positiva, negativa o neutral: {opinion}"

    zero_shot_prompt = f"Clasifica la siguiente opinión como positiva, negativa o neutral: {opinion}"


    few_shot_prompt = f"""
    Aquí tienes algunas opiniones clasificadas:

    Opinión: 'Me encanta este producto. Es increíble.' → Sentimiento: positivo
    Opinión: 'Este servicio es terrible, nunca lo recomendaría.' → Sentimiento: negativo
    Opinión: 'El artículo está bien, pero no es excepcional.' → Sentimiento: neutral

    Clasifica esta opinión: {opinion}
    """

    cot_prompt = f"""
    Pensando en la siguiente opinión:

    Opinión: '{opinion}'

    El sentimiento puede ser positivo, negativo o neutral. ¿Qué opinas?
    """


    prompts = {
        "prompt": prompt_base,
        "zero-shot": zero_shot_prompt,
        "few-shot": few_shot_prompt,
        "chain-of-thought": cot_prompt
    }

    # Generar predicción con Zero-shot
    prompt_zero_shot = zero_shot_prompt.format(opinion)
    result_zero_shot = generator(prompt_zero_shot, max_length=100, num_return_sequences=1)
    sentiment_zero_shot = result_zero_shot[0]["generated_text"]
    
    
    # Generar predicción con Few-shot
    prompt_few_shot = few_shot_prompt.format(opinion)
    result_few_shot = generator(prompt_few_shot, max_new_tokens=50, num_return_sequences=1)
    sentiment_few_shot = result_few_shot[0]["generated_text"]

    # Generar predicción con el prompt base
    prompt = prompt_base.format(opinion)
    result_base = generator(prompt, max_length=100, num_return_sequences=1)
    sentiment_base = result_base[0]["generated_text"]
    
    # Generar predicción con Chain-of-thought
    prompt_cot = cot_prompt.format(opinion)
    result_cot = generator(prompt_cot, max_length=100, num_return_sequences=1)
    sentiment_cot = result_cot[0]["generated_text"]
    
    # Asignar sentimientos de acuerdo al texto generado
    def assign_sentiment(response):
        if "positivo" in response.lower():
            return "positivo"
        elif "negativo" in response.lower():
            return "negativo"
        elif "neutral" in response.lower():
            return "neutral"
        else:
            return "desconocido"
    
    # Asignar sentimientos a cada predicción
    predictions_base.append(assign_sentiment(sentiment_base))
    predictions_zero_shot.append(assign_sentiment(sentiment_zero_shot))
    predictions_few_shot.append(assign_sentiment(sentiment_few_shot))
    predictions_cot.append(assign_sentiment(sentiment_cot))

    # Imprimir resultados
    print(f"Opinión: {opinion}\n → Zero-shot Sentimiento: {predictions_zero_shot[-1]}")
    print(f" → Few-shot Sentimiento: {predictions_few_shot[-1]}")
    print(f" → Chain-of-thought Sentimiento: {predictions_cot[-1]}")
    print(f" → Base Sentimiento: Sentimiento: {predictions_base[-1]}\n")

# Añadir las predicciones al DataFrame
df['prompt_base'] = prompt_base
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar el DataFrame con los resultados
print(df[['opinion', 'label', 'prompt_base' ,'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------------,[{'generated_text': 'The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es un gran servicio) is a \xa0positive term. \xa0The word is used because of the \xa0negative value of the term. \xa0One of the principal problems in frases is that the frase is held by men with\xa0a deep dislike for the woman as a woman. \xa0You can find a woman who is willing to break the rule of the man if'}]
-----------------------------------------,The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es un gran servicio) is a  positive term.  The word is used because of the  negative value of the term.  One of the principal problems in frases is that the frase is held by men with a deep dislike for the woman as a woman.  You can find a woman who is willing to break the rule of the man if


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es un gran servicio
 → Zero-shot Sentimiento: neutral
 → Few-shot Sentimiento: positivo
 → Chain-of-thought Sentimiento: positivo
 → Base Sentimiento: Sentimiento: neutral



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

-----------------------------------------,[{'generated_text': 'The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es impresionante) is a vernacular term given to an ancient Greek noun that means the act or course of a single verb or action. It is a way to express a positive attitude, either of a physical nature or of a negative attitude. [6]\n\nfouches are used by the word façade to describe a person. Often used both literally and figur'}]
-----------------------------------------,The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es impresionante) is a vernacular term given to an ancient Greek noun that means the act or course of a single verb or action. It is a way to express a positive attitude, either of a physical nature or of a negative attitude. [6]

fouches are used by the word façade to describe a person. Often used both literally and figur


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es impresionante
 → Zero-shot Sentimiento: neutral
 → Few-shot Sentimiento: positivo
 → Chain-of-thought Sentimiento: positivo
 → Base Sentimiento: Sentimiento: neutral



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------------,[{'generated_text': 'The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es increíble) is a ______________/__(es) that refers to the most productive way of gathering energy. It means that you produce energy by your energy, producing energy within the Frase. The Frase is really a type of ______________ (el) with the most productive way of production, so the expression can refer to the ______________/___. E'}]
-----------------------------------------,The sentiment of the next frase would you say is positive, negative or neutral?: the frase (Es increíble) is a ______________/__(es) that refers to the most productive way of gathering energy. It means that you produce energy by your energy, producing energy within the Frase. The Frase is really a type of ______________ (el) with the most productive way of production, so the expression can refer to the ______________/___. E
Opinión: Es increíble
 → Zero-shot S

In [3]:
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for opinion in df['opinion']:
    sentiment = classifier(opinion)
    print(f"Opinión: {opinion} → Sentimiento: {sentiment[0]['label']}")


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Opinión: Es un gran servicio → Sentimiento: 5 stars
Opinión: Es regular → Sentimiento: 3 stars
Opinión: Es un producto útil → Sentimiento: 4 stars
Opinión: Es un fraude → Sentimiento: 1 star
Opinión: Me gusta mucho → Sentimiento: 5 stars
Opinión: Es un desastre → Sentimiento: 1 star
Opinión: Es un mal servicio → Sentimiento: 1 star
Opinión: Es un producto inútil → Sentimiento: 1 star
Opinión: No lo recomendaría → Sentimiento: 2 stars
Opinión: Me encantó el producto → Sentimiento: 5 stars
Opinión: No me gusta → Sentimiento: 2 stars
Opinión: Es fantástico → Sentimiento: 5 stars
Opinión: Me encanta → Sentimiento: 5 stars
Opinión: Es un buen producto → Sentimiento: 4 stars
Opinión: Es muy bueno → Sentimiento: 5 stars
Opinión: Es un mal servicio → Sentimiento: 1 star
Opinión: Es una pérdida de dinero → Sentimiento: 1 star
Opinión: No es nada especial → Sentimiento: 2 stars
Opinión: Es un producto normal → Sentimiento: 3 stars
Opinión: Te odio → Sentimiento: 1 star
Opinión: Es un buen servic

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load model and tokenizer
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create the pipeline
classifier = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Define prompt
opinion = "Es un producto normal"
prompt = f"Classify the sentiment of the following text as positive, negative, or neutral: {opinion}"

# Get prediction
output = classifier(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0


negative
